<a href="https://colab.research.google.com/github/KjelleJ/enkla-ai-experiment/blob/main/AIX_8_kluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kluster

---
# Ett enkelt exempel: Skapa 3 kluster med totalt 200 punkter
---

In [ ]:
# Skapa data-punkter till klustren
# random_state=0 ger samma punkter varje gång
from sklearn.datasets import make_blobs
X, _ = make_blobs(n_samples=200, n_features=2, centers=3, cluster_std=0.5,
    shuffle=True, random_state=0)
print(X.shape)
X

In [ ]:
# Plotta data-punkterna som finns i X
import matplotlib.pyplot as plt
plt.scatter(X[:, 0], X[:, 1], c='white', marker='o', edgecolor='black', s=50)
plt.title("3 kluster, 200 punkter")
plt.grid()
plt.xlabel('X1')
plt.ylabel('X2')
plt.tight_layout()
plt.show()

---
# k-means - bestäm kluster
---

In [ ]:
# Vilket kluster (av 3) tillhör datapunkterna i X?
# Vilka är centrumpunkterna?
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3, init='random', n_init=10, max_iter=300, tol=1e-04, random_state=0)
y_km = km.fit_predict(X)
print(y_km)
print(km.cluster_centers_)

In [ ]:
# Plotta klustren
# Kluster 0
plt.scatter(X[y_km == 0, 0], X[y_km == 0, 1],
    s=50, c='green', marker='o', edgecolor='black', label='Kluster 0')
# Kluster 1
plt.scatter(X[y_km == 1, 0], X[y_km == 1, 1], s=50,
            c='yellow', marker='o', edgecolor='black', label='Kluster 1')
# Kluster 2
plt.scatter(X[y_km == 2, 0], X[y_km == 2, 1], s=50, c='white',
 marker='o', edgecolor='black', label='Kluster 2')
# Centrum
plt.scatter(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1], s=250,
                marker='*', c='red', edgecolor='black', label='Centrum')
plt.legend(scatterpoints=1)
plt.grid()
plt.title("Kluster bestämda av k-means")
plt.tight_layout()
plt.show()

---
# kNN - närmsta grannar. Ger vilket kluster en ny punkt tillhör
---

In [ ]:
# Kolla i plottningen ovanför om resultatet är rimligt
from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=14)
# anpassa, y_km är label från k_means
clf.fit(X, y_km)
# gör en prediktion för en punkt
for i in range(-3, 5, 2):
  for j in range(-1, 6, 2):
    pkt = [[float(i), float(j)]]
    Z = clf.predict(pkt)
    print(f"punkt {pkt} tillhör kluster {Z[0]}")

---
# EMBEDDINGS (INBÄDDNINGAR)
###En inbäddning är en representation av något (ett ord, en bild,...) i form av en vektor. Här för ett antal bilder: 30 bilder + testbild
---

In [ ]:
# Hämta bilderna och spara dom på fil
import requests
def get_img_to_file(url, path):
  img_data = requests.get(url).content
  with open(path, 'wb') as handler:
      handler.write(img_data)

for i in range(1, 11):
  get_img_to_file("https://www.download.gubboit.se/class0/" + str(i) + ".jpg", "0_" + str(i) + ".jpg")
  get_img_to_file("https://www.download.gubboit.se/class1/" + str(i) + ".jpg", "1_" + str(i) + ".jpg")
  get_img_to_file("https://www.download.gubboit.se/class2/" + str(i) + ".jpg", "2_" + str(i) + ".jpg")
get_img_to_file("https://www.download.gubboit.se/test/" + "1.jpg", "t_" + "1.jpg")

In [ ]:
# Plotta bilderna - 10 granar, 10 stugor, 10 katter + testbild (hus)
import matplotlib.pyplot as plt
from pathlib import Path

for i in range(3):
    for j in range(1, 11):
        plt.figure(figsize=(3, 2))
        plt.axis('off')
        print(str(i) + "_" + str(j) + ".jpg")
        plt.imshow(plt.imread(str(i) + "_" + str(j) + ".jpg"))
        plt.show()
# testbild
print("TESTBILD")
plt.figure(figsize=(3, 2))
plt.axis('off')
plt.imshow(plt.imread('t_1.jpg'))
plt.show()

---
# Vi ska använda MobileNet som "embedding model"
---

Men först visar vi att MobileNet är en modell för bildigenkänning

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# Hämta MobileNetv2 - hela modellen
from tensorflow import keras

model = keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=True,
    input_shape=(160, 160, 3))

In [ ]:
model.summary()

**MobileNet** är tränat med dataset **ImageNet**.

Det finns 1000 klasser i ImageNet. En beskrivning av klasserna finns här:

https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/

In [ ]:
# Predict med vår modell. Testa med katt och gran.
# Det finns flera klasser för katter men ingen för gran.
pimage = plt.imread("2_1.jpg") # katt
#pimage = plt.imread("0_3.jpg") # gran
 # lägg till batch-dimension, normalisera till (0.0, 1.0)
pimage = pimage.astype(np.float32)[np.newaxis, ...] / 255.
# ändra storlek till 160x160
pimage = tf.image.resize(pimage, (160, 160))
np.argmax(model.predict(pimage))

---
#Skapa vektorer/inbäddningar för bilderna med hjälp av en inbäddningsmodell
---

Vi använder MobileNetV2 men nu utan topp för att kunna skapa inbäddningar

In [ ]:
# Hämta MobileNetv2 - nu utan topp för att kunna skapa inbäddningar
conv_base = keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(160, 160, 3))

In [ ]:
conv_base.summary()

### Definiera vår modell

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(160, 160, 3))
x = conv_base(inputs)
x = layers.MaxPooling2D(pool_size=5)(x)
outputs = layers.Flatten()(x)

model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

### Beräkna vektorerna för bilderna

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

k = 0
X = np.zeros((31, 1280))
for i in range(3):
    for j in range(1, 11):
        pimage = plt.imread(str(i) + "_" + str(j) + ".jpg")
        # lägg till batch-dimension, normalisera till (0.0, 1.0)
        pimage = pimage.astype(np.float32)[np.newaxis, ...] / 255.
        # ändra storlek till 160x160
        pimage = tf.image.resize(pimage, (160, 160))
        X[k,:] = model.predict(pimage, verbose=0)[0]
        k = k + 1
# Testbild
pimage = plt.imread("t_1.jpg")
 # lägg till batch-dimension, normalisera till (0.0, 1.0)
pimage = pimage.astype(np.float32)[np.newaxis, ...] / 255.
# ändra storlek till 160x160
pimage = tf.image.resize(pimage, (160, 160))
X[k,:] = model.predict(pimage)[0]

In [ ]:
# Varje vektor innehåller 1280 element. 31 vektorer i matrisen X.
X.shape

## Bestäm klustren med k-means

In [ ]:
# Bestäm klustren (0, 1 eller 2) för X:s datapunkter
# Om det funkar bör resultatet bli:
# [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 1]
# Rader i X:
# 0-9 granar
# 10-19 stugor
# 20-29 katter
# 30: stuga (test)
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3, init='random', n_init=10, max_iter=300, tol=1e-04, random_state=0)
y_km = km.fit_predict(X)
print(y_km)

## kNN - närmsta grannar

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors
# labels (klasser) för punkt 0-29 (från k-means tidigare)
y0_29 = y_km[:30]
# punkt 0-29
X0_29 = X[:30, :]
clf = neighbors.KNeighborsClassifier(n_neighbors=5)
# anpassa
clf.fit(X0_29, y0_29)
# gör en prediktion på punkt 30
Z = clf.predict(X[30:, :])
print(f"kNN säger kluster {Z[0]} för punkt 30")
print(f"k-means säger kluster {y_km[30]} för punkt 30")

## Visualisera

In [ ]:
# standardisera - förbättrar resultatet något
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_std = sc.fit_transform(X)

In [ ]:
# Gå från 1280 dimensioner till 2 dimensioner med hjälp av PCA
# Möjliggör plotning
from sklearn.decomposition import PCA
# init PCA transformer till 2d
pca = PCA(n_components=2)
# dimensionality reduction:
X_pca = pca.fit_transform(X_std)
X_pca.shape

In [ ]:
# Plotta punkterna i 2d
# 10 bilder av gran, 10 bilder av stuga, 10 bilder av katt
# testbild: stuga
import matplotlib.pyplot as plt
act_part = X_pca[:8]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='green', marker='x', s=50, label='gran')
act_part = X_pca[8:9]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='green', marker='*', edgecolor='green',s=100, label='julgran')
act_part = X_pca[9:10]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='green', marker='x', s=50,)
act_part = X_pca[10:20]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='red', marker='o', edgecolor='red',s=50, label='stuga')
act_part = X_pca[20:30]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='black', marker='o', edgecolor='black',s=50, label='katt')
act_part = X_pca[30:]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='red', marker='x', s=50, label='stuga (test)')
plt.legend(scatterpoints=1)
plt.grid()
plt.title("2d representation (från 1280d med PCA)")
plt.tight_layout()
plt.show()

In [ ]:
# Gå till 2d men utan PCA
X2 = np.zeros((31, 2))
X2[:, 0] = X[:, 400]
X2[:, 1] = X[:, 800]
X2.shape

In [ ]:
# 10 bilder av gran, 10 bilder av stuga, 10 bilder av katt
# testbild: stuga
import matplotlib.pyplot as plt
act_part = X2[:8]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='green', marker='x', s=50, label='gran')
act_part = X2[8:9]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='green', marker='*', edgecolor='green',s=100, label='julgran')
act_part = X2[9:10]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='green', marker='x',s=50,)
act_part = X2[10:20]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='red', marker='o', edgecolor='red',s=50, label='stuga')
act_part = X2[20:30]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='black', marker='o', edgecolor='black',s=50, label='katt')
act_part = X2[30:]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='red', marker='x',s=50, label='stuga (test)')
plt.legend(scatterpoints=1)
plt.grid()
plt.title("2d representation (från 1280d utan PCA)")
plt.tight_layout()
plt.show()